In [6]:
import pandas as pd
import numpy as np
import matplotlib
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegressionCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression

In [7]:
df = pd.read_csv('./data/test.csv')


In [8]:
# split cabin into num let
cabin_loc_let = []
cabin_loc_num = []

for i in df['Cabin']:
    x = str(i)
    cabin_loc_let.append(x[0])
    if x[1] == 'a':
        cabin_loc_num.append(0)
    else:
        cabin_loc_num.append(x[1:])
        
df['cabin_loc_let'] = cabin_loc_let
df['cabin_loc_num'] = cabin_loc_num

In [9]:
# label encoder
cll_le = preprocessing.LabelEncoder() 
cll_enc = cll_le.fit_transform(df['cabin_loc_let'])
sex_le = preprocessing.LabelEncoder() 
sex_enc = sex_le.fit_transform(df['Sex'])
pclass_le = preprocessing.LabelEncoder() 
pclass_enc = pclass_le.fit_transform(df['Pclass'])
embarked_le = preprocessing.LabelEncoder() 
embarked_enc = embarked_le.fit_transform(df['Embarked'])

In [10]:
# one hot Pclass, Embarked, Cabin, Ticket, Sex
enc = OneHotEncoder(sparse = True)

cll_enc = cll_enc.reshape(-1,1)
sex_enc = sex_enc.reshape(-1,1)
pclass_enc = pclass_enc.reshape(-1,1)
embarked_enc = embarked_enc.reshape(-1,1)


cll_onehot = enc.fit_transform(cll_enc).toarray()
cll_cat = enc.categories_
sex_onehot = enc.fit_transform(sex_enc).toarray()
sex_cat = enc.categories_
pclass_onehot = enc.fit_transform(pclass_enc).toarray()
pclass_cat = enc.categories_
embarked_onehot = enc.fit_transform(embarked_enc).toarray()
embarked_cat = enc.categories_


In [11]:
# get column names
cll_cat = np.asarray(cll_cat)
sex_cat = np.asarray(sex_cat)
pclass_cat = np.asarray(pclass_cat)
embarked_cat = np.asarray(embarked_cat)

cll_cat = cll_cat.reshape(-1,1)
sex_cat = sex_cat.reshape(-1,1)
pclass_cat = pclass_cat.reshape(-1,1)
embarked_cat = embarked_cat.reshape(-1,1)

cll_inv = cll_le.inverse_transform(cll_cat)
sex_inv = sex_le.inverse_transform(sex_cat)
pclass_inv = pclass_le.inverse_transform(pclass_cat)
embarked_inv = embarked_le.inverse_transform(embarked_cat)


c:\users\jimmy\appdata\local\programs\python\python38\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [12]:
cll_df = pd.DataFrame(data = cll_onehot, columns = cll_inv)
sex_df = pd.DataFrame(data = sex_onehot, columns = sex_inv)
pclass_df = pd.DataFrame(data = pclass_onehot, columns = pclass_inv)
embarked_df = pd.DataFrame(data = embarked_onehot, columns = embarked_inv)

df2 = pd.concat([df, cll_df, sex_df, pclass_df, embarked_df], axis = 1)


In [13]:
df2 = df2.drop(['Ticket', 'Sex', 'Cabin', 'Embarked', 'Pclass', 'Name', 'cabin_loc_let'], axis = 1)

In [14]:
# replace nan
# df2.columns[df2.isna().any()].tolist()
mean_age = df2['Age'].mean()
mean_fare = df2['Fare'].mean()
df2['Age'] = df2['Age'].fillna(value = mean_age)
df2['Fare'] = df2['Fare'].fillna(value = mean_fare)


df2.columns[df2.isna().any()].tolist()



[]

In [15]:
# gbm model, iterate for params
from sklearn.ensemble import GradientBoostingClassifier
X_test = df2


In [16]:
# save model as file
# import pickle

# filename = 'clf10.sav'
# pickle.dump(clf10, open(filename, 'wb'))

# filename = 'clf100.sav'
# pickle.dump(clf100, open(filename, 'wb'))

# filename = 'clf500.sav'
# pickle.dump(clf500, open(filename, 'wb'))

# filename = 'clf1000.sav'
# pickle.dump(clf1000, open(filename, 'wb'))

In [17]:
# use model for test
import pickle

clf10 = pickle.load(open('./clf10.sav', 'rb'))
clf10_output = clf10.predict(X_test)

clf100 = pickle.load(open('./clf100.sav', 'rb'))
clf100_output = clf100.predict(X_test)

clf500 = pickle.load(open('./clf500.sav', 'rb'))
clf500_output = clf500.predict(X_test) 

clf1000 = pickle.load(open('./clf1000.sav', 'rb'))
clf1000_output = clf1000.predict(X_test) 


In [33]:
# output to data frame
clf10_sub = pd.DataFrame(clf10_output, columns = ['Survived'])
clf10_sub['PassengerId'] = range(100000, 200000)
clf10_sub = clf10_sub.reindex(columns = ['PassengerId', 'Survived'])
clf10_sub = clf10_sub.set_index(['PassengerId'])

clf100_sub = pd.DataFrame(clf100_output, columns = ['Survived'])
clf100_sub['PassengerId'] = range(100000, 200000)
clf100_sub = clf100_sub.reindex(columns = ['PassengerId', 'Survived'])
clf100_sub = clf100_sub.set_index(['PassengerId'])

clf500_sub = pd.DataFrame(clf500_output, columns = ['Survived'])
clf500_sub['PassengerId'] = range(100000, 200000)
clf500_sub = clf500_sub.reindex(columns = ['PassengerId', 'Survived'])
clf500_sub = clf500_sub.set_index(['PassengerId'])

clf1000_sub = pd.DataFrame(clf1000_output, columns = ['Survived'])
clf1000_sub['PassengerId'] = range(100000, 200000)
clf1000_sub = clf1000_sub.reindex(columns = ['PassengerId', 'Survived'])
clf1000_sub = clf1000_sub.set_index(['PassengerId'])



In [34]:
# to csv

clf10_sub.to_csv('./data/submissions/clf10_sub.csv', header = True)
clf100_sub.to_csv('./data/submissions/clf100_sub.csv', header = True)
clf500_sub.to_csv('./data/submissions/clf500_sub.csv', header = True)
clf1000_sub.to_csv('./data/submissions/clf1000_sub.csv', header = True)



In [ ]:
### clf 100 performed the best